In [26]:
!pip install requests requests-html pandas

# if python < python3.7
# pip install dataclasses

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 83 kB 1.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 105 kB 9.4 MB/s            
     |████████████████████████████████| 6.6 MB 6.9 MB/s            
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=dfff3fb5f4d64a4df651f9a014ce6cbebe2c93dd28c57a802a16a74d71282d2b
  Stored in directory: /home/jovyan/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=547c692499e9e0072cc6e289b31cc4b7a7a8c4b0b50a5d3f93287ebf147b19cf
  Stored in directory: /home/jovyan/.cache/pip/wheels/ae/e7/76/7dd44644d065268ab0e1b4fa2e802fa4bb0157717b7d6c6d92
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=cebc5a2c8aecae926b8bc368f592e3092c19a4bba0310d662b25

In [27]:
from dataclasses import dataclass
import pathlib
import pandas as pd
import requests
from requests_html import HTML

In [28]:
@dataclass
class ScrapeBoxOffice:
    base_endpoint:str = "https://www.boxofficemojo.com/year/world/"
    year:int = None
    save_raw:bool = False
    save:bool = False
    output_dir: str = "."
    table_selector: str = '.imdb-scroll-table'
    table_data = []
    table_header_names = []
    df = pd.DataFrame()
    
    @property
    def name(self):
        return self.year if isinstance(self.year, int) else 'world'
    
    def get_endpoint(self):
        endpoint = self.base_endpoint
        if isinstance(self.year, int):
            endpoint = f"{endpoint}{self.year}/"
        return endpoint
    
    def get_output_dir(self):
        return pathlib.Path(self.output_dir)
    
    def extract_html_str(self, endpoint=None):
        url = endpoint if endpoint is not None else self.get_endpoint()
        r = requests.get(url, stream=True)
        html_text = None
        status = r.status_code
        if r.status_code == 200:
            html_text = r.text
            if self.save_raw:
                output_fname = f"{self.name}.html"
                raw_output_dir = self.get_output_dir() / 'html'
                raw_output_dir.mkdir(exist_ok=True, parents=True)
                output_fname = raw_output_dir / output_fname
                with open(f"{output_fname}", 'w') as f:
                    f.write(html_text)
            return html_text, status
        return html_text, status
    
    def parse_html(self, html_str=''):
        r_html = HTML(html=html_str)
        r_table = r_html.find(self.table_selector)
        if len(r_table) == 0:
            return None
        table_data = []
        header_names = []
        parsed_table = r_table[0]
        rows = parsed_table.find("tr")
        header_row = rows[0]
        header_cols = header_row.find('th')
        header_names = [x.text for x in header_cols]
        for row in rows[1:]:
            cols = row.find("td")
            row_data = []
            row_dict_data = {}
            for i, col in enumerate(cols):
                header_name = header_names[i]
                row_data.append(col.text)
            table_data.append(row_data)
        self.table_data = table_data
        self.table_header_names = header_names
        return self.table_data, self.table_header_names
    
    def to_df(self, data=[], columns=[]):
        return pd.DataFrame(data, columns=columns)
    
    def run(self, save=False):
        save = self.save if save is False else save
        endpoint = self.get_endpoint()
        html_str, status = self.extract_html_str(endpoint=endpoint)
        if status not in range(200, 299):
            raise Exception(f"Extraction failed, endpoint status {status} at {endpoint}")
        data, headers = self.parse_html(html_str if html_str is not None else '')
        df = self.to_df(data=data, columns=headers)
        self.df = df
        if save:
            filepath = self.get_output_dir() / f'{self.name}.csv'
            df.to_csv(filepath, index=False)
        return self.df

In [32]:
for year in range(1985, 2023):
    scraper = ScrapeBoxOffice(year=year, save=True, save_raw=True, output_dir='box_office_mojo')
    df = scraper.run()
    df.head()